In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0" bokeh==2.0.1
!pip install autogluon --no-cache-dir

In [3]:
import pandas as pd
from autogluon.tabular import TabularPredictor

In [4]:
df_train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv',parse_dates=['datetime'])
df_test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv',parse_dates=['datetime'])
submission = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')

In [5]:
df_train.describe()

In [6]:
df_test.head()

In [7]:
submission.head()

In [8]:
df_train = df_train.drop(columns=['registered','casual'])

In [9]:
df_train.plot(kind='hist')

In [10]:
predictor = TabularPredictor(label='count', eval_metric='root_mean_squared_error').fit(train_data=df_train, presets='best_quality', time_limit=300)

In [11]:
summary = predictor.fit_summary()

In [12]:
predictor.leaderboard(silent=True).plot(kind='bar', x='model',y='score_val')

In [13]:
predictions = predictor.predict(df_test)

In [14]:
df_test['count'] = predictions
predictor.evaluate(df_test)

In [15]:
df_test.loc[predictions < 0,'count'] = 0

In [16]:
submission['count'] = predictions

In [17]:
submission.to_csv("submission.csv", index=False)

In [18]:
df_train['day'] = df_train.datetime.dt.day
df_train['month'] = df_train.datetime.dt.month
df_train['hour'] = df_train.datetime.dt.hour

df_test['day'] = df_test.datetime.dt.day
df_test['month'] = df_test.datetime.dt.month
df_test['hour'] = df_test.datetime.dt.hour

In [19]:
df_train["season"] = df_train["season"].astype('category')
df_train["weather"] = df_train["weather"].astype('category')
df_test["season"] = df_test["season"].astype('category')
df_test["weather"] = df_test["weather"].astype('category')

In [20]:
predictor_new_features = TabularPredictor(label='count', eval_metric='root_mean_squared_error').fit(train_data=df_train, presets='best_quality', time_limit=300)


In [21]:
predictor_new_features.fit_summary()

In [22]:
predictions = predictor_new_features.predict(df_test)

In [23]:
submissions_new_hpo = submission
submissions_new_hpo["count"] = predictions
submissions_new_hpo.loc[predictions < 0,"count"] = 0 

In [24]:
submission_new_hpo.to_csv("submission_new_hpo.csv", index=False)

In [ ]:
predictions

In [ ]:
submission_new_hpo

In [ ]:
params={
    'searcher':'rl',
    'scheduler':'rl',
    'use_orig_features': False,
    'max_base_models': 50,
    'max_base_models_per_type': 10,
    'save_bag_folds': True
}

In [ ]:
predictor_new_features = TabularPredictor(label='count', eval_metric='root_mean_squared_error').fit(train_data=df_train, presets='best_quality', time_limit=300,hyperparameter_tune_kwargs=params)
predictions = predictor_new_features.predict(df_test)

In [ ]:
predictor_new_features

In [ ]:
submissions_new_hpo = submission
submissions_new_hpo["count"] = predictions
submissions_new_hpo.loc[predictions < 0,"count"] = 0 